<a href="https://colab.research.google.com/github/matchbou/Tensorflow20-20190314/blob/master/04_deploy_and_distribute_tf2-Fin001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deploy and Distribute TensorFlow

In this notebook you will learn how to deploy TensorFlow models to TensorFlow Serving (TFS), using the REST API or the gRPC API, and how to train a model across multiple devices.

## Imports

In [0]:
!pip install tf-nightly-gpu-2.0-preview

In [0]:
%matplotlib inline

In [0]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
from tensorflow import keras
import time

In [0]:
print("python", sys.version)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

python 3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]
matplotlib 3.0.3
numpy 1.14.6
pandas 0.22.0
sklearn 0.20.3
tensorflow 2.0.0-dev20190313
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [0]:
assert sys.version_info >= (3, 5) # Python ≥3.5 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

![Exercise](https://c1.staticflickr.com/9/8101/8553474140_c50cf08708_b.jpg)

## Exercise 1 – Deploying a Model to TensorFlow Serving

## Save/Load a `SavedModel`

In [0]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.
X_test = X_test / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd",
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 7s 123us/sample - loss: 1.5023 - accuracy: 0.5617 - val_loss: 1.0624 - val_accuracy: 0.6814
Epoch 2/10
55000/55000 [==============================] - 6s 108us/sample - loss: 0.9437 - accuracy: 0.7017 - val_loss: 0.8335 - val_accuracy: 0.7342
Epoch 3/10
55000/55000 [==============================] - 6s 109us/sample - loss: 0.7964 - accuracy: 0.7431 - val_loss: 0.7378 - val_accuracy: 0.7656
Epoch 4/10
55000/55000 [==============================] - 6s 109us/sample - loss: 0.7215 - accuracy: 0.7689 - val_loss: 0.6800 - val_accuracy: 0.7848
Epoch 5/10
55000/55000 [==============================] - 6s 108us/sample - loss: 0.6724 - accuracy: 0.7839 - val_loss: 0.6390 - val_accuracy: 0.7970
Epoch 6/10
55000/55000 [==============================] - 6s 108us/sample - loss: 0.6371 - accuracy: 0.7933 - val_loss: 0.6090 - val_accuracy: 0.8064
Epoch 7/10
55000/55000 [===========================

In [0]:
MODEL_NAME = "my_fashion_mnist"
!rm -rf {MODEL_NAME}

In [0]:
import time

model_version = int(time.time())
model_path = os.path.join(MODEL_NAME, str(model_version))
os.makedirs(model_path)

In [0]:
tf.saved_model.save(model, model_path)

In [0]:
for root, dirs, files in os.walk(MODEL_NAME):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

my_fashion_mnist/
    1552556689/
        saved_model.pb
        assets/
        variables/
            variables.data-00000-of-00001
            variables.index


In [0]:
!saved_model_cli show --dir {model_path}

The given SavedModel contains the following tag-sets:
serve


In [0]:
!saved_model_cli show --dir {model_path} --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


In [0]:
!saved_model_cli show --dir {model_path} --tag_set serve \
                      --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [0]:
!saved_model_cli show --dir {model_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28)
        name: serving_default_flatten_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict


**Warning**: as you can see, the method name is empty. This is [a bug](https://github.com/tensorflow/tensorflow/issues/25235), hopefully it will be fixed shortly. In the meantime, you must use `keras.experimental.export()` instead of `tf.saved_model.save()`:

In [0]:
!rm -rf {MODEL_NAME}
model_path = keras.experimental.export(model, MODEL_NAME).decode("utf-8")
!saved_model_cli show --dir {model_path} --all

AttributeError: ignored

Let's write a few test instances to a `npy` file so we can pass them easily to our model:

In [0]:
X_new = X_test[:3]
np.save("my_fashion_mnist_tests.npy", X_new, allow_pickle=False)

In [0]:
input_name = model.input_names[0]
input_name

'flatten_input'

And now let's use `saved_model_cli` to make predictions for the instances we just saved:

In [0]:
!saved_model_cli run --dir {model_path} --tag_set serve \
                     --signature_def serving_default    \
                     --inputs {input_name}=my_fashion_mnist_tests.npy

Traceback (most recent call last):
  File "/usr/local/bin/saved_model_cli", line 10, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 909, in main
    args.func(args)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 643, in run
    init_tpu=args.init_tpu, tf_debug=args.tf_debug)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 303, in run_saved_model_with_feed_dict
    tag_set)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/tools/saved_model_utils.py", line 113, in get_meta_graph_def
    saved_model = read_saved_model(saved_model_dir)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/tools/saved_model_utils.py", line 55, in read_saved_model
    raise IOError("SavedModel file does not exist at: %s" % saved_model_dir)
OSError: SavedModel file does not exist at: my_fashion_mnist/155

## TensorFlow Serving

Install [Docker](https://docs.docker.com/install/) if you don't have it already. Then run:

```bash
docker pull tensorflow/serving

docker run -it --rm -p 8501:8501 \
   -v "`pwd`/my_fashion_mnist:/models/my_fashion_mnist" \
   -e MODEL_NAME=my_fashion_mnist \
   tensorflow/serving
```

Once you are finished using it, press Ctrl-C to shut down the server.

In [0]:
import json

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})
print(input_data_json[:200] + "..." + input_data_json[-200:])

{"signature_name": "serving_default", "instances": [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0,... 0.0, 0.3843137254901961, 0.6235294117647059, 0.2784313725490196, 0.0, 0.0, 0.26666666666666666, 0.6901960784313725, 0.6431372549019608, 0.22745098039215686, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]]}


Now let's use TensorFlow Serving's REST API to make predictions:

In [0]:
import requests

SERVER_URL = 'http://localhost:8501/v1/models/my_fashion_mnist:predict'
            
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status()
response = response.json()

ConnectionError: ignored

In [0]:
response.keys()

NameError: ignored

In [0]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

NameError: ignored

### Using Serialized Examples

In [0]:
serialized = []
for image in X_new:
    image_data = tf.train.FloatList(value=image.ravel())
    features = tf.train.Features(
        feature={
            "image": tf.train.Feature(float_list=image_data),
        }
    )
    example = tf.train.Example(features=features)
    serialized.append(example.SerializeToString())

In [0]:
[data[:100]+b'...' for data in serialized]

[b'\n\xd3\x18\n\xd0\x18\n\x05image\x12\xc6\x18\x12\xc3\x18\n\xc0\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...',
 b'\n\xd3\x18\n\xd0\x18\n\x05image\x12\xc6\x18\x12\xc3\x18\n\xc0\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xd1\xd0P=\x87\x86\x86>\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc9\xc8H>\x99\x98\x18>\x00\x00\x00\x00\x00\x00...',
 b'\n\xd3\x18\n\xd0\x18\n\x05image\x12\xc6\x18\x12\xc3\x18\n\xc0\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x81\x80\x80;\x00\x00\x00\x00\x87\x86\x86>\xb2\xb1

In [0]:
def parse_images(serialized):
    expected_features = {
        "image": tf.io.FixedLenFeature([28 * 28], dtype=tf.float32)
    }
    examples = tf.io.parse_example(serialized, expected_features)
    return tf.reshape(examples["image"], (-1, 28, 28))

In [0]:
parse_images(serialized)

<tf.Tensor: id=149282, shape=(3, 28, 28), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)>

In [0]:
serialized_inputs = keras.layers.Input(shape=[], dtype=tf.string)
images = keras.layers.Lambda(lambda serialized: parse_images(serialized))(serialized_inputs)
y_proba = model(images)
ser_model = keras.models.Model(inputs=[serialized_inputs], outputs=[y_proba])

In [0]:
SER_MODEL_NAME = "my_ser_fashion_mnist"
!rm -rf {SER_MODEL_NAME}
ser_model_path = keras.experimental.export(ser_model, SER_MODEL_NAME).decode("utf-8")
!saved_model_cli show --dir {ser_model_path} --all

AttributeError: ignored

```bash
docker run -it --rm -p 8500:8500 -p 8501:8501 \
   -v "`pwd`/my_ser_fashion_mnist:/models/my_ser_fashion_mnist" \
   -e MODEL_NAME=my_ser_fashion_mnist \
   tensorflow/serving
```

In [0]:
import base64
import json

ser_input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": [{"b64": base64.b64encode(data).decode("utf-8")}
                  for data in serialized],
})
print(ser_input_data_json[:200] + "..." + ser_input_data_json[-200:])

{"signature_name": "serving_default", "instances": [{"b64": "CtMYCtAYCgVpbWFnZRLGGBLDGArAGAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...7j4AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAxcTEPqCfHz+Pjo4+AAAAAAAAAACJiIg+sbAwP6WkJD/p6Gg+AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA="}]}


In [0]:
import requests

SER_SERVER_URL = 'http://localhost:8501/v1/models/my_ser_fashion_mnist:predict'
            
response = requests.post(SER_SERVER_URL, data=ser_input_data_json)
response.raise_for_status()
response = response.json()

ConnectionError: ignored

In [0]:
response.keys()

NameError: ignored

In [0]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

NameError: ignored

In [0]:
!python3 -m pip install --no-deps tensorflow-serving-api

In [0]:
import grpc
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)

request = predict_pb2.PredictRequest()
request.model_spec.name = SER_MODEL_NAME
request.model_spec.signature_name = "serving_default"
input_name = ser_model.input_names[0]
request.inputs[input_name].CopyFrom(tf.compat.v1.make_tensor_proto(serialized))

result = predict_service.Predict(request, 10.0)

_Rendezvous: ignored

In [0]:
result

NameError: ignored

In [0]:
output_name = ser_model.output_names[0]
output_name

'sequential'

In [0]:
shape = [dim.size for dim in result.outputs[output_name].tensor_shape.dim]
shape

NameError: ignored

In [0]:
y_proba = np.array(result.outputs[output_name].float_val).reshape(shape)
y_proba.round(2)

NameError: ignored

![Exercise](https://c1.staticflickr.com/9/8101/8553474140_c50cf08708_b.jpg)

## Exercise 2 – Distributed Training

In [0]:
keras.backend.clear_session()

In [0]:
distribution = tf.distribute.MirroredStrategy()

with distribution.scope():
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ])
    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd",
                  metrics=["accuracy"])

In [0]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), batch_size=25)

Epoch 1/10
2200/2200 [==============================] - 18s 8ms/step - loss: 1.4183 - accuracy: 0.5942 - val_loss: 0.9772 - val_accuracy: 0.6958
Epoch 2/10
2200/2200 [==============================] - 11s 5ms/step - loss: 0.8739 - accuracy: 0.7182 - val_loss: 0.7793 - val_accuracy: 0.7514
Epoch 3/10
2200/2200 [==============================] - 11s 5ms/step - loss: 0.7466 - accuracy: 0.7586 - val_loss: 0.6955 - val_accuracy: 0.7794
Epoch 4/10
2200/2200 [==============================] - 11s 5ms/step - loss: 0.6801 - accuracy: 0.7792 - val_loss: 0.6435 - val_accuracy: 0.7956
Epoch 5/10
2200/2200 [==============================] - 11s 5ms/step - loss: 0.6364 - accuracy: 0.7932 - val_loss: 0.6078 - val_accuracy: 0.8066
Epoch 6/10
2200/2200 [==============================] - 11s 5ms/step - loss: 0.6048 - accuracy: 0.8034 - val_loss: 0.5808 - val_accuracy: 0.8132
Epoch 7/10
2200/2200 [==============================] - 11s 5ms/step - loss: 0.5807 - accuracy: 0.8102 - val_loss: 0.5605 - val_ac